In [2]:
import pandas as pd

In [ ]:
car_df=pd.read_csv("D:\\Make_my_trip\\Recommendation\\cars.csv")
car_df.head()

,User_ID,travelCode,Check-in,pickupLocation,dropoffLocation,carType,rentalAgency,rentalDuration,Car_total_distance,fuelPolicy,Car_bookingStatus,total_rent_price
0,0,22,02/27/2020,Johntown,Port Brian,Sedan,Sixt,3,285,Prepaid,Pending,3410
1,0,45,08/06/2020,Owensland,Ruizfort,Sedan,Enterprise,3,412,Full-to-Full,Pending,4644
2,2,117,02/27/2020,Edwardview,Katiefort,Sedan,Hertz,1,433,Partial,Confirmed,4740
3,2,119,03/12/2020,Moranborough,Lake Stephanie,Hatchback,Budget,3,421,Full-to-Full,Confirmed,3744
4,2,153,11/05/2020,Port Kathrynstad,East Ronnieberg,Sedan,Enterprise,5,100,Prepaid,Cancelled,1694


In [10]:
sample_car=car_df.head().to_csv("D:\\Make_my_trip\\Recommendation\\sample_car.csv",index=False)

In [6]:
# load the entire dataset of flight, hotel and cars booking.
whole_ds=pd.read_csv("D:\\Make_my_trip\\2.csv")

In [7]:
whole_ds.head()

,travelCode,user_id,departure,arrival,flightType,flight_price,flight_duration,distance,agency,flight_date,...,pickupLocation,dropoffLocation,carType,rentalAgency,rentalDuration,totalDistance,fuelPolicy,bookingStatus,total_rent_price,total_price
0,0,0,Recife (PE),Florianopolis (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,09/26/2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2686.46
1,2,0,Aracaju (SE),Salvador (BH),firstClass,1684.05,2.16,830.86,CloudFy,10/10/2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2210.87
2,7,0,Aracaju (SE),Salvador (BH),economic,964.83,2.16,830.86,CloudFy,11/14/2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1755.06
3,11,0,Brasilia (DF),Salvador (BH),premium,1268.97,1.76,676.56,Rainbow,12/12/2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2322.61
4,13,0,Recife (PE),Florianopolis (SC),firstClass,1434.38,1.76,676.53,FlyingDrops,12/26/2019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1747.40


In [11]:
sample_whole=whole_ds.head().to_csv("D:\\Make_my_trip\\Recommendation\\sample_ds.csv",index=False)

In [8]:
whole_ds.columns

Index(['travelCode', 'user_id', 'departure', 'arrival', 'flightType',
       'flight_price', 'flight_duration', 'distance', 'agency', 'flight_date',
       'hotel_name', 'hotel_place', 'days', 'hotel_base', 'total_hotel',
       'check_in', 'company', 'user_name', 'gender', 'age', 'pickupLocation',
       'dropoffLocation', 'carType', 'rentalAgency', 'rentalDuration',
       'totalDistance', 'fuelPolicy', 'bookingStatus', 'total_rent_price',
       'total_price'],
      dtype='object')

following are the fields needed for recommendation:
1. user_id: Unique identifier for each user
2. booking_id: Unique booking identifier
3. destination: Trip destination
4. trip_duration: Number of days for the trip
5. num_travelers: Number of people traveling
6. flight_date: Date of flight booking
7. hotel_booked: Boolean indicating hotel booking
8. hotel_destination: Hotel destination
9. car_booked: Boolean indicating car booking
10. user_preferences: Optional field for additional user preferences

In [ ]:
required_fields=['user_id','travelCode',]# these are the fields that are to be kept from the whole_ds dataframe.

## Simple Car Recommendation
used Random Forest Model

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier  # Example model
from sklearn.metrics import accuracy_score

class TravelDataProcessor:
    def __init__(self, data_path):
        self.raw_data = pd.read_csv(data_path)
        self.processed_data = None
        self.model = None  # Initialize model here

    def preprocess_data(self):
        df = self.raw_data.copy()

        # Convert dates to datetime
        df['flight_date'] = pd.to_datetime(df['flight_date'])
        df['check_in'] = pd.to_datetime(df['check_in'])

        # Calculate check_out date
        df['check_out'] = df['check_in'] + pd.to_timedelta(df['days'], unit='D')

        # Engineer features relevant for car recommendations
        df['trip_duration'] = df['days']
        df['is_long_distance'] = df['distance'] > df['distance'].median()

        # Calculate potential car need features
        df['needs_airport_transfer'] = True  # Since all bookings include flights

        # Calculate price per day for reference
        df['price_per_day'] = df['total_price'] / df['days']

        # Create binary features
        df['is_business_class'] = df['flightType'].isin(['firstClass', 'premium'])
        df['is_expensive_hotel'] = df['hotel_base'] > df['hotel_base'].median()

        # Create potential car rental locations (can be improved)
        df['suggested_pickup_location'] = df['arrival']
        df['suggested_dropoff_location'] = df['arrival']

        # Suggest car rental duration (same as hotel stay)
        df['suggested_rental_duration'] = df['days']

        # Create feature for potential car recommendation (used for training initially)
        df['recommend_car'] = (
            (df['distance'] > 500) |
            (df['days'] >= 3) |
            (df['price_per_day'] > df['price_per_day'].median())
        )

        self.processed_data = df
        return df

    def train_car_recommendation_model(self):
        df = self.processed_data.copy()

        features = [
            'trip_duration',
            'is_long_distance',
            'is_business_class',
            'is_expensive_hotel',
            'price_per_day'
        ]

        X = df[features]
        y = df['recommend_car'].astype(int)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        self.model = RandomForestClassifier(random_state=42)
        self.model.fit(X_train, y_train)

        y_pred = self.model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Model Accuracy: {accuracy}")

    def generate_car_recommendations(self, user_id):
        user_data = self.processed_data[self.processed_data['user_id'] == user_id].copy()

        recommendations = []

        for _, booking in user_data.iterrows():
            features = [
                booking['trip_duration'],
                booking['is_long_distance'],
                booking['is_business_class'],
                booking['is_expensive_hotel'],
                booking['price_per_day']
            ]
            X_user = pd.DataFrame([features], columns=[
                'trip_duration', 'is_long_distance', 'is_business_class', 'is_expensive_hotel', 'price_per_day'
            ])

            if self.model is not None:  # Check if the model is trained
                should_recommend = self.model.predict(X_user)[0]
            else:
                should_recommend = False  # Or use a default rule if not trained

            if should_recommend:
                rec = {
                    'booking_id': booking['travelCode'],
                    'destination': booking['arrival'],
                    'recommended_car_type': self._suggest_car_type(booking),
                    'pickup_location': booking['arrival'],
                    'dropoff_location': booking['arrival'],
                    'suggested_duration': booking['days'],
                    'estimated_price': self._estimate_car_price(booking),
                    'recommendation_reason': self._get_recommendation_reason(booking) + " (Model Recommendation)"
                }
                recommendations.append(rec)

        return recommendations

    def _suggest_car_type(self, booking):
        # ... (Existing logic remains the same)
        if booking['flightType'] in ['firstClass', 'premium']:
            return 'Luxury'
        elif booking['total_price'] > self.processed_data['total_price'].median():
            return 'Premium'
        elif booking['days'] > 3:
            return 'Midsize'
        else:
            return 'Economy'

    def _estimate_car_price(self, booking):
        # ... (Existing logic remains the same)
        base_price_per_day = booking['hotel_base'] * 0.5
        return base_price_per_day * booking['days']

    def _get_recommendation_reason(self, booking):
        # ... (Existing logic remains the same)
        reasons = []

        if booking['distance'] > 500:
            reasons.append("Long distance travel")
        if booking['days'] >= 3:
            reasons.append("Extended stay duration")
        if booking['flightType'] in ['firstClass', 'premium']:
            reasons.append("Premium travel preference")

        return ", ".join(reasons) if reasons else "Convenience for local travel"


def main():
    processor = TravelDataProcessor('D:\\Make_my_trip\\Recommendation\\sample_ds.csv')  # Replace with your data path
    processed_data = processor.preprocess_data()

    processor.train_car_recommendation_model()  # Train the model

    recommendations = processor.generate_car_recommendations(0)  # User ID 0

    print("\nCar Rental Recommendations:")
    for rec in recommendations:
        print("\nBooking ID:", rec['booking_id'])
        print(f"Destination: {rec['destination']}")
        print(f"Recommended Car Type: {rec['recommended_car_type']}")
        print(f"Pickup Location: {rec['pickup_location']}")
        print(f"Dropoff Location: {rec['dropoff_location']}")
        print(f"Suggested Duration: {rec['suggested_duration']} days")
        print(f"Estimated Price: ${rec['estimated_price']:.2f}")
        print(f"Reason: {rec['recommendation_reason']}")

if __name__ == "__main__":
    main()

Model Accuracy: 1.0

Car Rental Recommendations:

Booking ID: 0
Destination: Florianopolis (SC)
Recommended Car Type: Luxury
Pickup Location: Florianopolis (SC)
Dropoff Location: Florianopolis (SC)
Suggested Duration: 4 days
Estimated Price: $626.04
Reason: Long distance travel, Extended stay duration, Premium travel preference (Model Recommendation)

Booking ID: 2
Destination: Salvador (BH)
Recommended Car Type: Luxury
Pickup Location: Salvador (BH)
Dropoff Location: Salvador (BH)
Suggested Duration: 2 days
Estimated Price: $263.41
Reason: Long distance travel, Premium travel preference (Model Recommendation)

Booking ID: 7
Destination: Salvador (BH)
Recommended Car Type: Economy
Pickup Location: Salvador (BH)
Dropoff Location: Salvador (BH)
Suggested Duration: 3 days
Estimated Price: $395.12
Reason: Long distance travel, Extended stay duration (Model Recommendation)

Booking ID: 11
Destination: Salvador (BH)
Recommended Car Type: Luxury
Pickup Location: Salvador (BH)
Dropoff Location

In [ ]:
cars=pd.read_excel(r"D:\Make_my_trip\FinalDataset\CarFINALdataset.xlsx")

In [7]:
cars.to_csv("carsfds.csv",index=False)

## Training and Comparing different ML models and evaluating each of the models performance.

In [2]:
###  comparing different ML models:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

class RecommendationProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.processed_data = None
        self.models = {}

    def preprocess_data(self):
        df = pd.read_csv(self.file_path)

        # Identify date columns and parse them
        date_columns = [col for col in ['flight_date', 'check_in', 'check_out'] if col in df.columns]
        for col in date_columns:
            df[col] = pd.to_datetime(df[col], errors='coerce', format='%d-%m-%Y')

        # Drop rows where any date is missing
        if date_columns:
            df.dropna(subset=date_columns, inplace=True)

        # Feature Engineering
        df['days'] = (df['check_out'] - df['check_in']).dt.days if 'check_out' in df.columns else 1
        df['distance'] = df['distance'].fillna(df['distance'].median()) if 'distance' in df.columns else 100
        df['price_per_day'] = df['price_per_day'].fillna(df['price_per_day'].median()) if 'price_per_day' in df.columns else 500

        # Create target variable with at least two classes
        df['recommend_car'] = (
            (df['distance'] > df['distance'].median() * 1.1) |  
            (df['days'] >= df['days'].median() * 0.9) |  
            (df['price_per_day'] > df['price_per_day'].median() * 1.1)
        ).astype(int)

        print(df['recommend_car'].value_counts())  # Debugging step to verify class balance
        if df['recommend_car'].nunique() == 1:
            df['recommend_car'] = np.random.choice([0, 1], size=len(df), p=[0.5, 0.5])

        self.processed_data = df

    def train_models(self):
        df = self.processed_data
        feature_columns = ['distance', 'days', 'price_per_day']
        X = df[[col for col in feature_columns if col in df.columns]]
        y = df['recommend_car']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        models = {
            'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
            'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
            'Decision Tree': DecisionTreeClassifier(random_state=42),
            'Logistic Regression': LogisticRegression(),
            'SVM': SVC()
        }

        best_model = None
        best_accuracy = 0

        for name, model in models.items():
            print(f"Training {name}...")
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            print(f"{name} Performance:")
            print(classification_report(y_test, y_pred))
            print(f"Accuracy: {accuracy:.4f}\n")

            self.models[name] = model
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = name

        print(f"Best Model: {best_model} with Accuracy: {best_accuracy:.4f}")


def main():
    processor = RecommendationProcessor("D:\\Make_my_trip\\previous_ds\\2.csv")
    processor.preprocess_data()
    processor.train_models()

if __name__ == "__main__":
    main()




recommend_car
1    16099
Name: count, dtype: int64
Training XGBoost...
XGBoost Performance:
              precision    recall  f1-score   support

           0       0.48      0.51      0.49      1537
           1       0.52      0.49      0.51      1683

    accuracy                           0.50      3220
   macro avg       0.50      0.50      0.50      3220
weighted avg       0.50      0.50      0.50      3220

Accuracy: 0.5019

Training Random Forest...


d:\Make_my_trip\Recommendation\recomm\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:56:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Random Forest Performance:
              precision    recall  f1-score   support

           0       0.48      0.53      0.51      1537
           1       0.53      0.48      0.50      1683

    accuracy                           0.50      3220
   macro avg       0.50      0.50      0.50      3220
weighted avg       0.51      0.50      0.50      3220

Accuracy: 0.5028

Training Decision Tree...
Decision Tree Performance:
              precision    recall  f1-score   support

           0       0.48      0.51      0.49      1537
           1       0.52      0.49      0.51      1683

    accuracy                           0.50      3220
   macro avg       0.50      0.50      0.50      3220
weighted avg       0.50      0.50      0.50      3220

Accuracy: 0.5019

Training Logistic Regression...
Logistic Regression Performance:
              precision    recall  f1-score   support

           0       0.48      0.77      0.59      1537
           1       0.52      0.23      0.32      1683

 

## Refining the performance of each ML model(pipeline):

In [3]:
## refining the models and evaluating their performance.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

class RecommendationProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.processed_data = None
        self.models = {}
        self.best_model = None

    def preprocess_data(self):
        df = pd.read_csv(self.file_path)
        date_columns = [col for col in ['flight_date', 'check_in', 'check_out'] if col in df.columns]
        for col in date_columns:
            df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
        if date_columns:
            df.dropna(subset=date_columns, inplace=True)
        
        df['days'] = (df['check_out'] - df['check_in']).dt.days if 'check_out' in df.columns else 1
        df['distance'] = df['distance'].fillna(df['distance'].median()) if 'distance' in df.columns else 100
        df['price_per_day'] = df['price_per_day'].fillna(df['price_per_day'].median()) if 'price_per_day' in df.columns else 500
        df['recommend_car'] = (
            (df['distance'] > df['distance'].median() * 1.1) |  
            (df['days'] >= df['days'].median() * 0.9) |  
            (df['price_per_day'] > df['price_per_day'].median() * 1.1)
        ).astype(int)
        
        if df['recommend_car'].nunique() == 1:
            df['recommend_car'] = np.random.choice([0, 1], size=len(df), p=[0.5, 0.5])
        
        self.processed_data = df

    def train_and_evaluate_models(self):
        df = self.processed_data
        feature_columns = ['distance', 'days', 'price_per_day']
        X = df[[col for col in feature_columns if col in df.columns]]
        y = df['recommend_car']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        models = {
            "XGBoost": XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss'),
            "Random Forest": RandomForestClassifier(n_estimators=200, max_depth=10, min_samples_split=5, random_state=42),
            "Decision Tree": DecisionTreeClassifier(max_depth=10, min_samples_split=5, random_state=42),
            "Logistic Regression": LogisticRegression(solver='liblinear', C=1.0),
            "SVM": SVC(kernel='rbf', C=1.0, gamma='scale')
        }

        best_accuracy = 0
        for name, model in models.items():
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            print(f"{name} Model Performance:")
            print(classification_report(y_test, y_pred))
            print(f"Accuracy: {accuracy:.4f}\n")
            self.models[name] = model
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                self.best_model = (name, model)
        
        print(f"Best Model: {self.best_model[0]} with Accuracy: {best_accuracy:.4f}")

def main():
    processor = RecommendationProcessor("D:\\Make_my_trip\\previous_ds\\2.csv")
    processor.preprocess_data()
    processor.train_and_evaluate_models()

if __name__ == "__main__":
    main()


C:\Users\hetvi\AppData\Local\Temp\ipykernel_1808\3160890175.py:23: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
C:\Users\hetvi\AppData\Local\Temp\ipykernel_1808\3160890175.py:23: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
d:\Make_my_trip\Recommendation\recomm\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:57:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Model Performance:
              precision    recall  f1-score   support

           0       0.49      0.41      0.45      2402
           1       0.52      0.60      0.56      2526

    accuracy                           0.51      4928
   macro avg       0.51      0.51      0.50      4928
weighted avg       0.51      0.51      0.50      4928

Accuracy: 0.5083

Random Forest Model Performance:
              precision    recall  f1-score   support

           0       0.49      0.41      0.45      2402
           1       0.52      0.60      0.56      2526

    accuracy                           0.51      4928
   macro avg       0.51      0.51      0.50      4928
weighted avg       0.51      0.51      0.50      4928

Accuracy: 0.5083

Decision Tree Model Performance:
              precision    recall  f1-score   support

           0       0.49      0.41      0.45      2402
           1       0.52      0.60      0.56      2526

    accuracy                           0.51      4928

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

class RecommendationProcessor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.processed_data = None
        self.models = {}

    def preprocess_data(self):
        df = pd.read_csv(self.file_path)
        date_columns = [col for col in ['flight_date', 'check_in', 'check_out'] if col in df.columns]
        
        for col in date_columns:
            df[col] = pd.to_datetime(df[col], errors='coerce', format='%d-%m-%Y')
        
        if date_columns:
            df.dropna(subset=date_columns, inplace=True)
        
        df['days'] = (df['check_out'] - df['check_in']).dt.days if 'check_out' in df.columns else 1
        df['distance'] = df['distance'].fillna(df['distance'].median()) if 'distance' in df.columns else 100
        df['price_per_day'] = df['price_per_day'].fillna(df['price_per_day'].median()) if 'price_per_day' in df.columns else 500
        
        df['recommend_car'] = (
            (df['distance'] > df['distance'].median() * 1.1) |  
            (df['days'] >= df['days'].median() * 0.9) |  
            (df['price_per_day'] > df['price_per_day'].median() * 1.1)
        ).astype(int)
        
        if df['recommend_car'].nunique() == 1:
            df['recommend_car'] = np.random.choice([0, 1], size=len(df), p=[0.5, 0.5])
        
        self.processed_data = df

    def train_models(self):
        df = self.processed_data
        feature_columns = ['distance', 'days', 'price_per_day']
        X = df[[col for col in feature_columns if col in df.columns]]
        y = df['recommend_car']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        models = {
            'XGBoost': XGBClassifier(),
            'RandomForest': RandomForestClassifier(),
            'SVM': SVC(),
            'KNN': KNeighborsClassifier()
        }
        
        tuned_params = {
            'XGBoost': {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.2]},
            'RandomForest': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]},
            'SVM': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']},
            'KNN': {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}
        }
        
        for name, model in models.items():
            print(f"Training {name}...")
            search = GridSearchCV(model, tuned_params[name], cv=5, scoring='accuracy', n_jobs=-1)
            search.fit(X_train, y_train)
            best_model = search.best_estimator_
            y_pred = best_model.predict(X_test)
            
            print(f"{name} Model Performance:")
            print(classification_report(y_test, y_pred))
            print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}\n")
            
            self.models[name] = best_model

def main():
    processor = RecommendationProcessor("D:\\Make_my_trip\\previous_ds\\2.csv")
    processor.preprocess_data()
    processor.train_models()

if __name__ == "__main__":
    main()


Training XGBoost...
XGBoost Model Performance:
              precision    recall  f1-score   support

           0       0.50      0.36      0.42      1597
           1       0.51      0.65      0.57      1623

    accuracy                           0.51      3220
   macro avg       0.50      0.50      0.49      3220
weighted avg       0.50      0.51      0.49      3220

Accuracy: 0.5053

Training RandomForest...
RandomForest Model Performance:
              precision    recall  f1-score   support

           0       0.50      0.38      0.43      1597
           1       0.51      0.63      0.56      1623

    accuracy                           0.50      3220
   macro avg       0.50      0.50      0.50      3220
weighted avg       0.50      0.50      0.50      3220

Accuracy: 0.5043

Training SVM...
SVM Model Performance:
              precision    recall  f1-score   support

           0       0.49      0.17      0.25      1597
           1       0.50      0.82      0.62      1623

   

# Using content based filtering for recommendation:

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
file_path = "D:/Make_my_trip/previous_ds/2.csv"
df = pd.read_csv(file_path)

# Ensure 'user_id' column exists
if 'user_id' not in df.columns:
    raise ValueError("The dataset must have a 'user_id' column.")

# Filter users who booked flights and hotels but NOT a car
users_without_cars = df[(df['flight_price'].notna()) & 
                        (df['total_hotel'].notna()) & 
                        (df['carType'].isna())].copy().reset_index(drop=True)

# Users who have booked a car (used for recommendations)
users_with_cars = df[df['carType'].notna()].copy().reset_index(drop=True)

# Fill missing values in feature columns
feature_cols = ['pickupLocation', 'dropoffLocation', 'rentalAgency', 'fuelPolicy']
users_with_cars[feature_cols] = users_with_cars[feature_cols].fillna("Unknown")
users_without_cars[feature_cols] = users_without_cars[feature_cols].fillna("Unknown")

# Combine feature columns into a single text feature
users_with_cars['combined_features'] = users_with_cars[feature_cols].astype(str).agg(' '.join, axis=1)
users_without_cars['combined_features'] = users_without_cars[feature_cols].astype(str).agg(' '.join, axis=1)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix_with_cars = tfidf.fit_transform(users_with_cars['combined_features'])
tfidf_matrix_without_cars = tfidf.transform(users_without_cars['combined_features'])

# Compute similarity between users who need cars and those who booked cars
cosine_sim = cosine_similarity(tfidf_matrix_without_cars, tfidf_matrix_with_cars)

# Function to recommend Top-N similar cars for a user
def recommend_car(user_id, top_n=3):
    if user_id not in df['user_id'].values:
        return f"User {user_id} does not exist in the dataset."

    if user_id in users_with_cars['user_id'].values:
        return f"User {user_id} has already booked a car."

    # Find the user index in users_without_cars
    user_row = users_without_cars[users_without_cars['user_id'] == user_id]
    
    if user_row.empty:
        return f"User {user_id} is not eligible for car recommendations."

    user_index = user_row.index[0]

    # Find the Top-N most similar users who booked a car
    top_indices = cosine_sim[user_index].argsort()[-top_n:][::-1]  # Get Top-N matches
    
    # Get recommended cars and remove duplicates
    recommended_cars = users_with_cars.iloc[top_indices][['carType', 'rentalAgency']].drop_duplicates().values

    return recommended_cars.tolist()

# Evaluate recommendation system
def evaluate_recommendations():
    correct_recommendations = 0
    total_recommendations = 0
    unique_recommended_cars = set()

    for user_id in users_without_cars['user_id'].unique():
        recommended_cars = recommend_car(user_id, top_n=3)

        if isinstance(recommended_cars, str):  # Skip if no recommendations
            continue

        recommended_cars = {tuple(car) for car in recommended_cars}  # Convert to set of tuples
        total_recommendations += len(recommended_cars)
        unique_recommended_cars.update(recommended_cars)

        # Corrected comparison: Check if both carType and rentalAgency match
        for car in recommended_cars:
            if any((users_with_cars['carType'] == car[0]) & (users_with_cars['rentalAgency'] == car[1])):
                correct_recommendations += 1

    precision = correct_recommendations / total_recommendations if total_recommendations else 0
    recall = correct_recommendations / len(users_with_cars) if len(users_with_cars) else 0
    coverage = len(unique_recommended_cars) / len(users_with_cars[['carType', 'rentalAgency']].drop_duplicates()) if len(users_with_cars) else 0

    return f"\n🔹 **Precision:** {precision:.2f}\n🔹 **Recall:** {recall:.2f}\n🔹 **Coverage:** {coverage:.2f}"

# Example: Recommend cars for a user
user_id = 5  # Change this to any user ID you want to check
print(f"Recommended Cars for User {user_id}: {recommend_car(user_id)}")

# Print evaluation metrics
print(evaluate_recommendations())


Recommended Cars for User 5: [['Sedan', 'Sixt'], ['Hatchback', 'Enterprise']]

🔹 **Precision:** 1.00
🔹 **Recall:** 0.11
🔹 **Coverage:** 0.10


In [10]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
file_path = "D:/Make_my_trip/previous_ds/2.csv"
df = pd.read_csv(file_path)

# Filter users who booked flights and hotels but NOT a car
users_without_cars = df[(df['flight_price'].notna()) & 
                        (df['total_hotel'].notna()) & 
                        (df['carType'].isna())].copy()

# Users who have booked a car (used for recommendations)
users_with_cars = df[df['carType'].notna()].copy()

# Fill missing values in feature columns using .loc to avoid warnings
feature_cols = ['pickupLocation', 'dropoffLocation', 'rentalAgency', 'fuelPolicy']
users_with_cars.loc[:, feature_cols] = users_with_cars[feature_cols].fillna("Unknown")
users_without_cars.loc[:, feature_cols] = users_without_cars[feature_cols].fillna("Unknown")

# Combine feature columns into a single text feature
users_with_cars.loc[:, 'combined_features'] = users_with_cars[feature_cols].astype(str).agg(' '.join, axis=1)
users_without_cars.loc[:, 'combined_features'] = users_without_cars[feature_cols].astype(str).agg(' '.join, axis=1)

# Reset index to avoid index mismatches
users_without_cars.reset_index(drop=True, inplace=True)
users_with_cars.reset_index(drop=True, inplace=True)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix_with_cars = tfidf.fit_transform(users_with_cars['combined_features'])
tfidf_matrix_without_cars = tfidf.transform(users_without_cars['combined_features'])

# Compute similarity between users who need cars and those who booked cars
cosine_sim = cosine_similarity(tfidf_matrix_without_cars, tfidf_matrix_with_cars)

# Function to recommend a car for a specific user (Top-N)
def recommend_car(user_id, top_n=3):
    if user_id not in users_without_cars['user_id'].values:
        return f"User {user_id} has already booked a car or does not exist in the dataset."

    # Get user index in users_without_cars
    user_index = users_without_cars[users_without_cars['user_id'] == user_id].index[0]

    # Ensure index is within valid range
    if user_index >= cosine_sim.shape[0]:
        return f"Error: User index {user_index} is out of range."

    # Find the Top-N most similar users who booked a car
    top_indices = cosine_sim[user_index].argsort()[-top_n:][::-1]

    # Ensure indices are valid
    top_indices = [idx for idx in top_indices if idx < len(users_with_cars)]

    # Get recommended cars
    recommended_cars = users_with_cars.iloc[top_indices][['carType', 'rentalAgency']].values

    return recommended_cars if recommended_cars.size > 0 else "No suitable recommendations found."

# Function to evaluate recommendations
def evaluate_recommendations(top_n=3):
    total_users = len(users_without_cars)
    successful_recommendations = 0
    total_recommendations = 0
    unique_recommended_cars = set()
    unique_actual_cars = set(users_with_cars['carType'].unique())

    for user_id in users_without_cars['user_id'].unique():
        top_cars = recommend_car(user_id, top_n)

        if isinstance(top_cars, str):  # Skip if no recommendations
            continue

        total_recommendations += len(top_cars)
        unique_recommended_cars.update([car[0] for car in top_cars])

        # Check if any recommended car matches the actual booked cars
        if any(car[0] in unique_actual_cars for car in top_cars):
            successful_recommendations += 1

    # Compute evaluation metrics
    precision = successful_recommendations / total_recommendations if total_recommendations > 0 else 0
    recall = successful_recommendations / total_users if total_users > 0 else 0
    coverage = len(unique_recommended_cars) / len(unique_actual_cars) if len(unique_actual_cars) > 0 else 0

    return f"""
    🔹 **Precision:** {precision:.2f}
    🔹 **Recall:** {recall:.2f}
    🔹 **Coverage:** {coverage:.2f}
    """

# Example: Recommend a car for user with ID 5
user_id = 5
top_n = 10  # Change this to Top-N recommendations
top_cars = recommend_car(user_id, top_n)
print(f"Recommended Cars for User {user_id}: {top_cars}")

# Print evaluation metrics
print(evaluate_recommendations(top_n=10))

Recommended Cars for User 5: [['Sedan' 'Sixt']
 ['Hatchback' 'Enterprise']
 ['Sedan' 'Sixt']
 ['Luxury' 'Enterprise']
 ['Sedan' 'Hertz']
 ['Sedan' 'Enterprise']
 ['Hatchback' 'Enterprise']
 ['Hatchback' 'Enterprise']
 ['Sedan' 'Hertz']
 ['SUV' 'Enterprise']]

    🔹 **Precision:** 0.10
    🔹 **Recall:** 0.04
    🔹 **Coverage:** 1.00
    


### recommendation considering the travelCode:

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
file_path = "D:/Make_my_trip/previous_ds/2.csv"
df = pd.read_csv(file_path)

# Filter users who booked flights and hotels but NOT a car
users_without_cars = df[(df['flight_price'].notna()) & 
                        (df['total_hotel'].notna()) & 
                        (df['carType'].isna())].copy()

# Users who have booked a car (used for recommendations)
users_with_cars = df[df['carType'].notna()].copy()

# Fill missing values in feature columns
feature_cols = ['pickupLocation', 'dropoffLocation', 'rentalAgency', 'fuelPolicy']
users_with_cars[feature_cols] = users_with_cars[feature_cols].fillna("Unknown")
users_without_cars[feature_cols] = users_without_cars[feature_cols].fillna("Unknown")

# Add travelCode to the combined feature set
users_with_cars['combined_features'] = users_with_cars[feature_cols + ['travelCode']].astype(str).agg(' '.join, axis=1)
users_without_cars['combined_features'] = users_without_cars[feature_cols + ['travelCode']].astype(str).agg(' '.join, axis=1)

# Reset index
users_without_cars.reset_index(drop=True, inplace=True)
users_with_cars.reset_index(drop=True, inplace=True)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix_with_cars = tfidf.fit_transform(users_with_cars['combined_features'])
tfidf_matrix_without_cars = tfidf.transform(users_without_cars['combined_features'])

# Compute similarity between users who need cars and those who booked cars
cosine_sim = cosine_similarity(tfidf_matrix_without_cars, tfidf_matrix_with_cars)

# Function to recommend a car with fallback option
def recommend_car(user_id, travel_code, top_n=3):
    user_record = users_without_cars[(users_without_cars['user_id'] == user_id) & (users_without_cars['travelCode'] == travel_code)]
    if user_record.empty:
        return f"User {user_id} with Travel Code {travel_code} has already booked a car or does not exist in the dataset."

    user_index = user_record.index[0]
    if user_index >= cosine_sim.shape[0]:
        return f"Error: User index {user_index} is out of range."

    # Find top-N most similar users who booked a car (exact travelCode match first)
    top_indices = cosine_sim[user_index].argsort()[-top_n:][::-1]
    travel_matches = users_with_cars.iloc[top_indices]
    travel_matches = travel_matches[travel_matches['travelCode'] == travel_code]
    
    if not travel_matches.empty:
        return travel_matches[['carType', 'rentalAgency']].values
    
    # Fallback: Recommend based on location & rental agency if no exact travelCode match
    location_matches = users_with_cars.iloc[top_indices]
    return location_matches[['carType', 'rentalAgency']].values if not location_matches.empty else "No suitable recommendations found."

# Example: Recommend a car for a specific user & travelCode
user_id = 0
travel_code = 2  # Replace with actual travel code from dataset
top_n = 10  # Change this to Top-N recommendations
top_cars = recommend_car(user_id, travel_code, top_n)
print(f"Recommended Cars for User {user_id} (Travel Code {travel_code}): {top_cars}")


Recommended Cars for User 0 (Travel Code 2): [['Sedan' 'Sixt']
 ['Hatchback' 'Enterprise']
 ['Sedan' 'Sixt']
 ['Luxury' 'Enterprise']
 ['Sedan' 'Hertz']
 ['Sedan' 'Enterprise']
 ['Hatchback' 'Enterprise']
 ['Hatchback' 'Enterprise']
 ['Sedan' 'Hertz']
 ['SUV' 'Enterprise']]


In [19]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
file_path = "D:/Make_my_trip/previous_ds/2.csv"
df = pd.read_csv(file_path)

# Filter users who booked flights and hotels but NOT a car
users_without_cars = df[(df['flight_price'].notna()) & 
                        (df['total_hotel'].notna()) & 
                        (df['carType'].isna())].copy()

# Users who have booked a car (used for recommendations)
users_with_cars = df[df['carType'].notna()].copy()

# Fill missing values in feature columns
feature_cols = ['pickupLocation', 'dropoffLocation', 'rentalAgency', 'fuelPolicy']
users_with_cars[feature_cols] = users_with_cars[feature_cols].fillna("Unknown")
users_without_cars[feature_cols] = users_without_cars[feature_cols].fillna("Unknown")

# Combine feature columns for similarity matching
users_with_cars['combined_features'] = users_with_cars[feature_cols + ['travelCode']].astype(str).agg(' '.join, axis=1)
users_without_cars['combined_features'] = users_without_cars[feature_cols + ['travelCode']].astype(str).agg(' '.join, axis=1)

# Reset index
users_without_cars.reset_index(drop=True, inplace=True)
users_with_cars.reset_index(drop=True, inplace=True)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix_with_cars = tfidf.fit_transform(users_with_cars['combined_features'])
tfidf_matrix_without_cars = tfidf.transform(users_without_cars['combined_features'])

# Compute cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix_without_cars, tfidf_matrix_with_cars)

# Function to recommend cars
def recommend_car(user_id, travel_code, top_n=3):
    user_record = users_without_cars[(users_without_cars['user_id'] == user_id) & (users_without_cars['travelCode'] == travel_code)]
    if user_record.empty:
        return f"User {user_id} with Travel Code {travel_code} has already booked a car or does not exist in the dataset."
    
    user_index = user_record.index[0]
    top_indices = cosine_sim[user_index].argsort()[-top_n:][::-1]
    recommended_cars = users_with_cars.iloc[top_indices]
    recommended_cars = recommended_cars[recommended_cars['travelCode'] == travel_code][['carType', 'rentalAgency']].values
    
    # If no exact match, find similar travel codes
    if recommended_cars.size == 0:
        travel_code_similarity = cosine_similarity(tfidf_matrix_with_cars, tfidf_matrix_with_cars)
        similar_travel_codes = users_with_cars.iloc[travel_code_similarity[user_index].argsort()[-5:][::-1]]['travelCode'].unique()
        
        recommended_cars = users_with_cars[users_with_cars['travelCode'].isin(similar_travel_codes)][['carType', 'rentalAgency']].values
    
    # If still no match, recommend most popular cars
    if recommended_cars.size == 0:
        popular_cars = users_with_cars.groupby(['carType', 'rentalAgency']).size().reset_index(name='count')
        recommended_cars = popular_cars.sort_values(by='count', ascending=False).head(top_n)[['carType', 'rentalAgency']].values
    
    return recommended_cars if recommended_cars.size > 0 else "No suitable recommendations found."

# Function to evaluate recommendations
def evaluate_recommendations(top_n=3):
    total_users = len(users_without_cars)
    successful_recommendations = 0
    total_recommendations = 0
    unique_recommended_cars = set()
    unique_actual_cars = set(users_with_cars['carType'].unique())

    for _, row in users_without_cars.iterrows():
        user_id, travel_code = row['user_id'], row['travelCode']
        top_cars = recommend_car(user_id, travel_code, top_n)
        if isinstance(top_cars, str):  # Skip if no recommendations
            continue
        total_recommendations += len(top_cars)
        unique_recommended_cars.update([car[0] for car in top_cars])
        if any(car[0] in unique_actual_cars for car in top_cars):
            successful_recommendations += 1
    
    precision = successful_recommendations / total_recommendations if total_recommendations > 0 else 0
    recall = successful_recommendations / total_users if total_users > 0 else 0
    coverage = len(unique_recommended_cars) / len(unique_actual_cars) if len(unique_actual_cars) > 0 else 0

    return f"""
    🔹 **Precision:** {precision:.2f}
    🔹 **Recall:** {recall:.2f}
    🔹 **Coverage:** {coverage:.2f}
    """

# Example: Recommend a car for a user
user_id = 0  # Replace with an actual user ID
travel_code = 2  # Replace with actual travel code
top_n = 10

top_cars = recommend_car(user_id, travel_code, top_n)
print(f"Recommended Cars for User {user_id} (Travel Code {travel_code}): {top_cars}")

# Evaluate system
print(evaluate_recommendations(top_n=10))


Recommended Cars for User 0 (Travel Code 2): [['Sedan' 'Enterprise']
 ['Hatchback' 'Enterprise']
 ['SUV' 'Enterprise']
 ['Luxury' 'Enterprise']
 ['SUV' 'Avis']
 ['Luxury' 'Sixt']
 ['SUV' 'Enterprise']]


KeyboardInterrupt: 

In [28]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
file_path = "D:/Make_my_trip/previous_ds/2.csv"
df = pd.read_csv(file_path)

# Filter users who booked flights and hotels but NOT a car
users_without_cars = df[(df['flight_price'].notna()) & 
                        (df['total_hotel'].notna()) & 
                        (df['carType'].isna())].copy()

# Users who have booked a car (used for recommendations)
users_with_cars = df[df['carType'].notna()].copy()

# Fill missing values in feature columns
# feature_cols = ['pickupLocation', 'dropoffLocation', 'rentalAgency', 'fuelPolicy']
feature_cols = ['pickupLocation', 'dropoffLocation']

users_with_cars[feature_cols] = users_with_cars[feature_cols].fillna("Unknown")
users_without_cars[feature_cols] = users_without_cars[feature_cols].fillna("Unknown")

# Add travelCode to the combined feature set
users_with_cars['combined_features'] = users_with_cars[feature_cols + ['travelCode']].astype(str).agg(' '.join, axis=1)
users_without_cars['combined_features'] = users_without_cars[feature_cols + ['travelCode']].astype(str).agg(' '.join, axis=1)

# Reset index
users_without_cars.reset_index(drop=True, inplace=True)
users_with_cars.reset_index(drop=True, inplace=True)

# common_travel_codes = set(users_without_cars['travelCode']).intersection(set(users_with_cars['travelCode']))
# print("Common Travel Codes:", common_travel_codes)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix_with_cars = tfidf.fit_transform(users_with_cars['combined_features'])
tfidf_matrix_without_cars = tfidf.transform(users_without_cars['combined_features'])

# Precompute cosine similarity between all users
cosine_sim = cosine_similarity(tfidf_matrix_without_cars, tfidf_matrix_with_cars)

# Group users_with_cars by travelCode for faster lookup
cars_by_travel_code = users_with_cars.groupby('travelCode')

# Convert DataFrame to NumPy arrays for faster lookups
user_ids_without_cars = users_without_cars['user_id'].to_numpy()
travel_codes_without_cars = users_without_cars['travelCode'].to_numpy()

# Function to recommend a car for a specific user (Top-N) per travelCode
def recommend_car(user_id, travel_code, top_n=3):
    user_record = users_without_cars[users_without_cars['user_id'] == user_id]

    if user_record.empty:
        print(f"❌ No entry found for User {user_id}.")
        return "No suitable recommendations found."

    # Find similar users (ignoring exact travelCode match)
    user_index = user_record.index[0]
    top_indices = cosine_sim[user_index].argsort()[-top_n:][::-1]

    recommended_cars = users_with_cars.iloc[top_indices][['carType', 'rentalAgency']].values

    if recommended_cars.size == 0:
        print(f"⚠️ No cars found for similar users.")
    
    return recommended_cars if recommended_cars.size > 0 else "No suitable recommendations found."

    # user_indices = np.where((user_ids_without_cars == user_id) & (travel_codes_without_cars == travel_code))[0]
    
    # if len(user_indices) == 0:
    #     return f"User {user_id} with Travel Code {travel_code} has already booked a car or does not exist in the dataset."

    # user_index = user_indices[0]

    # # Find the Top-N most similar users who booked a car
    # top_indices = cosine_sim[user_index].argsort()[-top_n:][::-1]
    # recommended_cars = users_with_cars.iloc[top_indices][['carType', 'rentalAgency']].values
    # # Get recommended cars for this travelCode
    # if travel_code in cars_by_travel_code.groups:
    #     recommended_cars = users_with_cars.iloc[top_indices]
    #     recommended_cars = recommended_cars[recommended_cars['travelCode'] == travel_code][['carType', 'rentalAgency']].values
    #     return recommended_cars if recommended_cars.size > 0 else "No suitable recommendations found."
    
    # return "No suitable recommendations found."


# Function to evaluate recommendations
def evaluate_recommendations(top_n=3):
    total_users = len(users_without_cars)
    successful_recommendations = 0
    total_recommendations = 0
    unique_recommended_cars = set()
    unique_actual_cars = set(users_with_cars['carType'].unique())

    for user_index, (user_id, travel_code) in enumerate(zip(user_ids_without_cars, travel_codes_without_cars)):
        top_cars = recommend_car(user_id, travel_code, top_n)

        if isinstance(top_cars, str):  # Skip if no recommendations
            continue

        total_recommendations += len(top_cars)
        unique_recommended_cars.update([car[0] for car in top_cars])

        # Check if any recommended car matches the actual booked cars
        if any(car[0] in unique_actual_cars for car in top_cars):
            successful_recommendations += 1

        # Print progress every 100 users
        if user_index % 100 == 0:
            print(f"Processed {user_index}/{total_users} users...")

    # Compute evaluation metrics
    precision = successful_recommendations / total_recommendations if total_recommendations > 0 else 0
    recall = successful_recommendations / total_users if total_users > 0 else 0
    coverage = len(unique_recommended_cars) / len(unique_actual_cars) if len(unique_actual_cars) > 0 else 0

    return f"""
    🔹 **Precision:** {precision:.2f}
    🔹 **Recall:** {recall:.2f}
    🔹 **Coverage:** {coverage:.2f}
    """

# Example: Recommend a car for a specific user & travelCode
user_id = 0
travel_code = 39  # Replace with actual travel code from dataset
top_n = 10  # Change this to Top-N recommendations
top_cars = recommend_car(user_id, travel_code, top_n)
print(f"Recommended Cars for User {user_id} (Travel Code {travel_code}): {top_cars}")

# Print evaluation metrics
print(evaluate_recommendations(top_n=10))


Recommended Cars for User 0 (Travel Code 39): [['Sedan' 'Sixt']
 ['Hatchback' 'Enterprise']
 ['Sedan' 'Sixt']
 ['Luxury' 'Enterprise']
 ['Sedan' 'Hertz']
 ['Sedan' 'Enterprise']
 ['Hatchback' 'Enterprise']
 ['Hatchback' 'Enterprise']
 ['Sedan' 'Hertz']
 ['SUV' 'Enterprise']]
Processed 0/36779 users...
Processed 100/36779 users...
Processed 200/36779 users...
Processed 300/36779 users...
Processed 400/36779 users...
Processed 500/36779 users...
Processed 600/36779 users...
Processed 700/36779 users...
Processed 800/36779 users...
Processed 900/36779 users...
Processed 1000/36779 users...
Processed 1100/36779 users...
Processed 1200/36779 users...
Processed 1300/36779 users...
Processed 1400/36779 users...
Processed 1500/36779 users...
Processed 1600/36779 users...
Processed 1700/36779 users...
Processed 1800/36779 users...
Processed 1900/36779 users...
Processed 2000/36779 users...
Processed 2100/36779 users...
Processed 2200/36779 users...
Processed 2300/36779 users...
Processed 2400/3

In [32]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
file_path = "D:/Make_my_trip/previous_ds/2.csv"
df = pd.read_csv(file_path)

# Filter users who booked flights and hotels but NOT a car
users_without_cars = df[(df['flight_price'].notna()) & 
                        (df['total_hotel'].notna()) & 
                        (df['carType'].isna())].copy()

# Users who have booked a car (used for recommendations)
users_with_cars = df[df['carType'].notna()].copy()

# Fill missing values in feature columns
feature_cols = ['pickupLocation', 'dropoffLocation', 'rentalAgency', 'fuelPolicy']
# feature_cols = ['pickupLocation', 'dropoffLocation']

users_with_cars[feature_cols] = users_with_cars[feature_cols].fillna("Unknown")
users_without_cars[feature_cols] = users_without_cars[feature_cols].fillna("Unknown")

# Add travelCode to the combined feature set
users_with_cars['combined_features'] = users_with_cars[feature_cols + ['travelCode']].astype(str).agg(' '.join, axis=1)
users_without_cars['combined_features'] = users_without_cars[feature_cols + ['travelCode']].astype(str).agg(' '.join, axis=1)

# Reset index
users_without_cars.reset_index(drop=True, inplace=True)
users_with_cars.reset_index(drop=True, inplace=True)

# common_travel_codes = set(users_without_cars['travelCode']).intersection(set(users_with_cars['travelCode']))
# print("Common Travel Codes:", common_travel_codes)

# TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix_with_cars = tfidf.fit_transform(users_with_cars['combined_features'])
tfidf_matrix_without_cars = tfidf.transform(users_without_cars['combined_features'])

# Precompute cosine similarity between all users
cosine_sim = cosine_similarity(tfidf_matrix_without_cars, tfidf_matrix_with_cars)

# Group users_with_cars by travelCode for faster lookup
cars_by_travel_code = users_with_cars.groupby('travelCode')

# Convert DataFrame to NumPy arrays for faster lookups
user_ids_without_cars = users_without_cars['user_id'].to_numpy()
travel_codes_without_cars = users_without_cars['travelCode'].to_numpy()

# Function to recommend a car for a specific user (Top-N) per travelCode
def recommend_car(user_id, travel_code, top_n=3):
    user_record = users_without_cars[users_without_cars['user_id'] == user_id]

    if user_record.empty:
        print(f"❌ No entry found for User {user_id}.")
        return "No suitable recommendations found."

    # Find similar users (ignoring exact travelCode match)
    user_index = user_record.index[0]
    top_indices = cosine_sim[user_index].argsort()[-top_n:][::-1]

    recommended_cars = users_with_cars.iloc[top_indices][['carType', 'rentalAgency']].values

    if recommended_cars.size == 0:
        print(f"⚠️ No cars found for similar users.")
    
    return recommended_cars if recommended_cars.size > 0 else "No suitable recommendations found."

# Function to evaluate recommendations
def evaluate_recommendations(top_n=3):
    total_users = len(users_without_cars)
    successful_recommendations = 0
    total_recommendations = 0
    unique_recommended_cars = set()
    unique_actual_cars = set(users_with_cars['carType'].unique())

    for user_index, (user_id, travel_code) in enumerate(zip(user_ids_without_cars, travel_codes_without_cars)):
        top_cars = recommend_car(user_id, travel_code, top_n)

        if isinstance(top_cars, str):  # Skip if no recommendations
            continue

        total_recommendations += len(top_cars)
        unique_recommended_cars.update([car[0] for car in top_cars])

        # Check if any recommended car matches the actual booked cars
        if any(car[0] in unique_actual_cars for car in top_cars):
            successful_recommendations += 1

        # # Print progress every 1000 users
        # if user_index % 1000 == 0:
        #     print(f"Processed {user_index}/{total_users} users...")

    # Compute evaluation metrics
    precision = successful_recommendations / total_recommendations if total_recommendations > 0 else 0
    recall = successful_recommendations / total_users if total_users > 0 else 0
    coverage = len(unique_recommended_cars) / len(unique_actual_cars) if len(unique_actual_cars) > 0 else 0

    return f"""
    🔹 **Precision:** {precision:.2f}
    🔹 **Recall:** {recall:.2f}
    🔹 **Coverage:** {coverage:.2f}
    """

# Example: Recommend a car for a specific user & travelCode
user_id = 0
travel_code = 22  # Replace with actual travel code from dataset
top_n = 10  # Change this to Top-N recommendations
top_cars = recommend_car(user_id, travel_code, top_n)
print(f"Recommended Cars for User {user_id} (Travel Code {travel_code}): {top_cars}")

# Print evaluation metrics
print(evaluate_recommendations(top_n=10))


Recommended Cars for User 0 (Travel Code 22): [['Sedan' 'Sixt']
 ['Hatchback' 'Enterprise']
 ['Sedan' 'Sixt']
 ['Luxury' 'Enterprise']
 ['Sedan' 'Hertz']
 ['Sedan' 'Enterprise']
 ['Hatchback' 'Enterprise']
 ['Hatchback' 'Enterprise']
 ['Sedan' 'Hertz']
 ['SUV' 'Enterprise']]

    🔹 **Precision:** 0.10
    🔹 **Recall:** 1.00
    🔹 **Coverage:** 1.00
    


In [7]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Load dataset
file_path = "D:/Make_my_trip/previous_ds/2.csv"  # Replace with your file path
df = pd.read_csv(file_path)

# Data preprocessing (same as before)
users_without_cars = df[(df['flight_price'].notna()) & 
                        (df['total_hotel'].notna()) & 
                        (df['carType'].isna())].copy()

users_with_cars = df[df['carType'].notna()].copy()

feature_cols = ['pickupLocation', 'dropoffLocation', 'rentalAgency', 'fuelPolicy']
users_with_cars[feature_cols] = users_with_cars[feature_cols].fillna("Unknown")
users_without_cars[feature_cols] = users_without_cars[feature_cols].fillna("Unknown")

users_with_cars['combined_features'] = users_with_cars[feature_cols + ['travelCode']].astype(str).agg(' '.join, axis=1)
users_without_cars['combined_features'] = users_without_cars[feature_cols + ['travelCode']].astype(str).agg(' '.join, axis=1)

users_without_cars.reset_index(drop=True, inplace=True)
users_with_cars.reset_index(drop=True, inplace=True)


# Split data into training and testing sets
train_users_with_cars, test_users_with_cars = train_test_split(users_with_cars, test_size=0.2, random_state=42)
train_users_without_cars, test_users_without_cars = train_test_split(users_without_cars, test_size=0.2, random_state=42)

# --- Improved TF-IDF Vectorization ---
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), min_df=2)  # Tuned parameters
tfidf_matrix_with_cars = tfidf.fit_transform(train_users_with_cars['combined_features'])
tfidf_matrix_without_cars = tfidf.transform(test_users_without_cars['combined_features'])


# Precompute cosine similarity between all users (on the training set)
cosine_sim = cosine_similarity(tfidf_matrix_without_cars, tfidf_matrix_with_cars)

# Recommendation function (FIXED)
def recommend_car(user_id, travel_code, top_n=3):
    user_record = test_users_without_cars[test_users_without_cars['user_id'] == user_id]

    if user_record.empty:
        print(f"❌ No entry found for User {user_id}.")
        return "No suitable recommendations found."

    user_index_tfidf = np.where(test_users_without_cars['user_id'] == user_id)[0][0]

    top_indices = cosine_sim[user_index_tfidf].argsort()[-top_n:][::-1]

    recommended_cars = train_users_with_cars.iloc[top_indices][['carType', 'rentalAgency']].values

    if recommended_cars.size == 0:
        print(f"⚠️ No cars found for similar users.")

    return recommended_cars if recommended_cars.size > 0 else "No suitable recommendations found."


# Evaluation function (modified for train/test split)
def evaluate_recommendations(top_n=3):
    total_users = len(test_users_without_cars)
    successful_recommendations = 0
    total_recommendations = 0
    unique_recommended_cars = set()
    unique_actual_cars = set(train_users_with_cars['carType'].unique())

    user_ids_without_cars = test_users_without_cars['user_id'].to_numpy()
    travel_codes_without_cars = test_users_without_cars['travelCode'].to_numpy()

    for user_index, (user_id, travel_code) in enumerate(zip(user_ids_without_cars, travel_codes_without_cars)):
        top_cars = recommend_car(user_id, travel_code, top_n)

        if isinstance(top_cars, str):
            continue

        total_recommendations += len(top_cars)
        unique_recommended_cars.update([car[0] for car in top_cars])

        actual_cars = train_users_with_cars[train_users_with_cars['travelCode'] == travel_code]['carType'].values
        if len(actual_cars) > 0:
            if any(car[0] in actual_cars for car in top_cars):
                successful_recommendations += 1

    precision = successful_recommendations / total_recommendations if total_recommendations > 0 else 0
    recall = successful_recommendations / total_users if total_users > 0 else 0
    coverage = len(unique_recommended_cars) / len(unique_actual_cars) if len(unique_actual_cars) > 0 else 0

    return f"""
    🔹 **Precision:** {precision:.2f}
    🔹 **Recall:** {recall:.2f}
    🔹 **Coverage:** {coverage:.2f}
    """

# Example and Evaluation (on the TEST set)
example_user_id = test_users_without_cars['user_id'].iloc[0]
example_travel_code = test_users_without_cars['travelCode'].iloc[0]
top_n_example = 10
example_recommendations = recommend_car(example_user_id, example_travel_code, top_n_example)
print(f"Recommended Cars for User {example_user_id} (Travel Code {example_travel_code}): {example_recommendations}")


print(evaluate_recommendations(top_n=10))

Recommended Cars for User 1104 (Travel Code 111548): [['Hatchback' 'Avis']
 ['Luxury' 'Hertz']
 ['Hatchback' 'Enterprise']
 ['Sedan' 'Hertz']
 ['Hatchback' 'Enterprise']
 ['Luxury' 'Enterprise']
 ['Hatchback' 'Budget']
 ['Luxury' 'Sixt']
 ['Sedan' 'Avis']
 ['Luxury' 'Sixt']]

    🔹 **Precision:** 0.00
    🔹 **Recall:** 0.00
    🔹 **Coverage:** 0.75
    


##### Separating the dataset into two csv files one with the users who have booked a car and the other with the user who did not book the car

In [34]:

# Load the original dataset
file_path = "D:/Make_my_trip/previous_ds/2.csv"
df = pd.read_csv(file_path)

# Separate users
users_with_cars = df[df['carType'].notna()]  # Users who booked a car
users_without_cars = df[df['carType'].isna()]  # Users who did NOT book a car

# Save them as CSV files
users_with_cars.to_csv("users_with_cars.csv", index=False)
users_without_cars.to_csv("users_without_cars.csv", index=False)

print("✅ Users successfully separated into two datasets!")


✅ Users successfully separated into two datasets!


Speed optimization:

In [ ]:
#pip install datasketch



[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip



  Obtaining dependency information for datasketch from https://files.pythonhosted.org/packages/8d/24/c8b0570c17c64e9d00485ac6f325c3a7ba19ea8b3385c73c85a26a519d77/datasketch-1.6.5-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/89.2 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/89.2 kB ? eta -:--:--
   --------- ------------------------------ 20.5/89.2 kB 165.2 kB/s eta 0:00:01
   --------- ------------------------------ 20.5/89.2 kB 165.2 kB/s eta 0:00:01
   --------- ------------------------------ 20.5/89.2 kB 165.2 kB/s eta 0:00:01
   -------------------------------- ------- 71.7/89.2 kB 302.7 kB/s eta 0:00:01
   -------------------------------- ------- 71.7/89.2 kB 302.7 kB/s eta 0:00:01
   ---------------------------------------- 89.2/89.2 kB 266.0 kB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datasketch import MinHash, MinHashLSH
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score

# Load datasets efficiently
users_with_cars = pd.read_csv("users_with_cars.csv", low_memory=False)
users_without_cars = pd.read_csv("users_without_cars.csv", low_memory=False)

# Fill missing values
users_with_cars.fillna("Unknown", inplace=True)
users_without_cars.fillna("Unknown", inplace=True)

# Feature Engineering
def create_combined_features(df):
    return (
        df["pickupLocation"].astype(str) + " " +
        df["dropoffLocation"].astype(str) + " " +
        df["rentalAgency"].astype(str) + " " +
        df["flightType"].astype(str) + " " +
        df["carType"].astype(str) + " " +
        df["travelCode"].astype(str)
    )

users_with_cars["combined_features"] = create_combined_features(users_with_cars)
users_without_cars["combined_features"] = create_combined_features(users_without_cars)

# ✅ **Fast Hashing Vectorizer Instead of TF-IDF**
vectorizer = HashingVectorizer(n_features=500, alternate_sign=False)
hash_matrix_with_cars = vectorizer.fit_transform(users_with_cars["combined_features"])
hash_matrix_without_cars = vectorizer.transform(users_without_cars["combined_features"])

# ✅ **Efficient Cosine Similarity Calculation**
cosine_sim = cosine_similarity(hash_matrix_without_cars, hash_matrix_with_cars)

# ✅ **MinHash for Fast Jaccard Similarity**
def get_minhash(text, num_perm=128):
    """Generate MinHash for a given text"""
    m = MinHash(num_perm=num_perm)
    for word in text.split():
        m.update(word.encode("utf8"))
    return m

# Build MinHash LSH for fast similarity search
lsh = MinHashLSH(threshold=0.5, num_perm=128)
minhash_dict = {}

for i, text in enumerate(users_with_cars["combined_features"]):
    mh = get_minhash(text)
    lsh.insert(i, mh)
    minhash_dict[i] = mh

# Calculate Jaccard Similarity using MinHash
jaccard_sim = np.zeros((len(users_without_cars), len(users_with_cars)))

for i, text in enumerate(users_without_cars["combined_features"]):
    mh = get_minhash(text)
    similar_indices = lsh.query(mh)  # Get similar users fast
    for j in similar_indices:
        jaccard_sim[i, j] = mh.jaccard(minhash_dict[j])

# ✅ **Hybrid Similarity Calculation**
final_similarity = (0.9 * cosine_sim) + (0.1 * jaccard_sim)

# ✅ **Optimized Recommendation Extraction (NumPy argpartition)**
recommendations = {}
top_k = 3

for i, user_id in enumerate(users_without_cars["user_id"]):
    top_indices = np.argpartition(final_similarity[i], -top_k)[-top_k:]
    top_indices = top_indices[np.argsort(final_similarity[i][top_indices])[::-1]]
    recommended_cars = users_with_cars.iloc[top_indices]["carType"].tolist()
    recommendations[user_id] = recommended_cars if recommended_cars else ["No suitable recommendations."]

# ✅ **Fast Decision Tree Instead of RandomForest**
label_encoder = LabelEncoder()
users_with_cars["carType_encoded"] = label_encoder.fit_transform(users_with_cars["carType"].astype(str))

X = users_with_cars[["travelCode", "rentalDuration", "totalDistance"]].astype(float)
y = users_with_cars["carType_encoded"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = DecisionTreeClassifier(max_depth=5, random_state=42)  # Shallow tree for fast prediction
model.fit(X_train, y_train)

# ✅ **Evaluation**
def evaluate_recommendations():
    y_pred = model.predict(X_test)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
    coverage = len(set(y_pred)) / max(len(set(y_test)), 1)
    return precision, recall, coverage

# ✅ **Display Results (Limit to 10 Users for Speed)**
print("\n🔹 Recommended Cars for Users:")
for user_id, cars in list(recommendations.items())[:10]:
    print(f"User {user_id}: {cars}")

precision, recall, coverage = evaluate_recommendations()

print("\n📊 **Evaluation Metrics:**")
print(f"🔹 **Precision:** {precision:.2f}")
print(f"🔹 **Recall:** {recall:.2f}")
print(f"🔹 **Coverage:** {coverage:.2f}")


C:\Users\hetvi\AppData\Local\Temp\ipykernel_16436\4224198477.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  users_without_cars.fillna("Unknown", inplace=True)



🔹 Recommended Cars for Users:
User 0: ['Hatchback', 'Sedan', 'Luxury']
User 1: ['Luxury', 'Sedan', 'SUV']
User 2: ['Luxury', 'SUV', 'Sedan']
User 3: ['SUV', 'Sedan', 'Luxury']
User 4: ['Luxury', 'SUV', 'Sedan']
User 5: ['Hatchback', 'Sedan', 'SUV']
User 6: ['Luxury', 'Sedan', 'SUV']
User 7: ['Luxury', 'SUV', 'Sedan']
User 8: ['SUV', 'Luxury', 'Sedan']
User 9: ['Luxury', 'Sedan', 'SUV']

📊 **Evaluation Metrics:**
🔹 **Precision:** 0.24
🔹 **Recall:** 0.24
🔹 **Coverage:** 1.00


following are the changes that are done in the new code below:

✅ Feature Engineering Improvements

✅ Weighted Hybrid Similarity (using optimal weights)

✅ One-Hot Encoding for categorical features

✅ Using XGBoost for better predictions

✅ Efficient Filtering of Recommendations

In [2]:
    import pandas as pd
    import numpy as np
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.preprocessing import LabelEncoder
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import precision_score, recall_score

    def preprocess_data(file_path):
        df = pd.read_csv(file_path)
        df.fillna("Unknown", inplace=True)  # Ensure missing values are handled correctly
        return df

    def create_combined_features(df):
        return (
            df["pickupLocation"].astype(str) + " " +
            df["dropoffLocation"].astype(str) + " " +
            df["rentalAgency"].astype(str) + " " +
            df["flightType"].astype(str) + " " +
            df["carType"].astype(str) + " " +
            df["travelCode"].astype(str)
        )

    def recommend_car_for_user(target_user_id, users_with_cars, users_without_cars):
        users_with_cars["combined_features"] = create_combined_features(users_with_cars)
        users_without_cars["combined_features"] = create_combined_features(users_without_cars)
        
        tfidf = TfidfVectorizer(max_features=500, stop_words='english')
        tfidf_matrix_with_cars = tfidf.fit_transform(users_with_cars["combined_features"])
        tfidf_matrix_without_cars = tfidf.transform(users_without_cars["combined_features"])
        
        cosine_sim = cosine_similarity(tfidf_matrix_without_cars, tfidf_matrix_with_cars)
        
        target_index = users_without_cars[users_without_cars["user_id"] == target_user_id].index[0]
        
        top_indices = np.argsort(cosine_sim[target_index])[::-1][:3]  # Top-3 recommendations
        recommended_cars = users_with_cars.iloc[top_indices]["carType"].tolist()
        
        return recommended_cars if recommended_cars else "No suitable recommendations found."

    def train_ml_model(users_with_cars):
        label_encoder = LabelEncoder()
        users_with_cars["carType_encoded"] = label_encoder.fit_transform(users_with_cars["carType"].astype(str))
        
        X = users_with_cars[["travelCode", "rentalDuration", "totalDistance"]]
        y = users_with_cars["carType_encoded"]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        model = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        coverage = len(set(y_pred)) / len(set(y_test))
        
        return precision, recall, coverage

    # Load data
    users_with_cars = preprocess_data("users_with_cars.csv")
    users_without_cars = preprocess_data("users_without_cars.csv")

    # Choose a user to recommend a car for
    target_user_id = 0  # Change this to any specific user_id
    recommended_cars = recommend_car_for_user(target_user_id, users_with_cars, users_without_cars)

    # Train ML model and evaluate
    precision, recall, coverage = train_ml_model(users_with_cars)

    # Display results
    print(f"Recommended Cars for User {target_user_id}: {recommended_cars}")
    print("\nEvaluation Metrics:")
    print(f"🔹 Precision: {precision:.2f}")
    print(f"🔹 Recall: {recall:.2f}")
    print(f"🔹 Coverage: {coverage:.2f}")


C:\Users\hetvi\AppData\Local\Temp\ipykernel_6456\2703936123.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("Unknown", inplace=True)  # Ensure missing values are handled correctly


Recommended Cars for User 0: ['Sedan', 'Sedan', 'Sedan']

Evaluation Metrics:
🔹 Precision: 0.23
🔹 Recall: 0.24
🔹 Coverage: 1.00


# new 

In [8]:
import pandas as pd
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
import numpy as np
from sklearn.metrics import precision_score, recall_score

# Load dataset
file_path = "D:/Make_my_trip/previous_ds/2.csv"  # Replace with your path
df = pd.read_csv(file_path)

# Data Preprocessing
# Ensure the required columns are present: 'user_id' and 'carType' 
df = df.dropna(subset=['user_id', 'carType'])

# Map carType to category codes for efficient processing
df['carType_id'] = df['carType'].astype('category').cat.codes

# Create user-item interaction matrix
interaction_data = df[['user_id', 'carType_id']].copy()
interaction_data['interaction'] = 1  # Implicit interaction feedback

user_item_matrix = coo_matrix((
    interaction_data['interaction'], 
    (interaction_data['user_id'], interaction_data['carType_id'])
))

# Initialize and Train ALS Model
model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=30)
model.fit(user_item_matrix)

# Recommendation for a user
def recommend_cars(user_id, top_n=5):
    user_interaction = user_item_matrix.getrow(user_id)
    recommended = model.recommend(user_id, user_interaction, N=top_n)
    
    car_id_mapping = df[['carType', 'carType_id']].drop_duplicates()
    car_id_to_name = car_id_mapping.set_index('carType_id')['carType'].to_dict()
    
    recommended_cars = [car_id_to_name[car_id] for car_id, _ in recommended]
    return recommended_cars

# Example usage
example_user_id = df['user_id'].iloc[0]  # Replace with a specific user ID if available
print(f"Recommended cars for user {example_user_id}: {recommend_cars(example_user_id)}")

# Evaluation
def evaluate_als_model(test_data, top_n=5):
    y_true = []
    y_pred = []
    
    for user_id in test_data['user_id'].unique():
        actual_car_types = test_data[test_data['user_id'] == user_id]['carType_id'].values
        predicted_cars = recommend_cars(user_id, top_n)
        
        if not predicted_cars or len(actual_car_types) == 0:
            continue
        
        # Map recommended car names to IDs
        predicted_car_ids = [df[df['carType'] == car]['carType_id'].values[0] for car in predicted_cars if car in df['carType'].unique()]
        
        # Update ground truth and predictions
        y_true.append(list(actual_car_types))
        y_pred.append(predicted_car_ids)
    
    # Flatten lists for precision and recall
    y_true_flat = [item for sublist in y_true for item in sublist]
    y_pred_flat = [item for sublist in y_pred for item in sublist]
    
    # Compute evaluation metrics
    precision = precision_score(y_true_flat, y_pred_flat, average='macro')
    recall = recall_score(y_true_flat, y_pred_flat, average='macro')
    
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

# Split test data
test_users_with_cars = df.sample(frac=0.2, random_state=42)
evaluate_als_model(test_users_with_cars)


ModuleNotFoundError: No module named 'implicit'